In [6]:
#Set Path to mapped_links. This file is precalculated by apping the links to each other.
#Set target path name for the output as link_ids_near_precip_csv     
#(This is the output, this will all the links mapped with the given maximum perpendicular distance)
#Set the precipitation device list path to [precip_devices_csv_file]

In [23]:
import pandas as pd
import numpy as np
from numpy import ones,vstack
from numpy.linalg import lstsq
import math
import csv
import geopy.distance
import geopy.point

In [42]:
root_folder = "/home/gayan/Desktop/CMLARE/Data/Master Files/"
mapped_links_csv_file = root_folder +  "mapped_links.csv"
link_ids_near_precip_csv = root_folder + "mapped_links_precip"
precip_devices_csv_file = root_folder + "precipitaion_stations_curw.csv"
mapped_links_field_names = ["cell_tower_id1","cell_tower_id2","source1_name","source2_name","device_name_1","device_name_2",
                            "frequency1","frequency2","frequency_band","latitude1","longitude1","latitude2","longitude2",
                            "district1","district2","id_1","id_2"]
formatted_links_field_names = ["cell_tower_id1","cell_tower_id2","source1_name","source2_name", "district1","district2",
                            "latitude1","latitude2","longitude1","longitude2","nearest_precip_station","distance",
                            "nearest_precip_lat","nearest_precip_long","id_1","id_2","PathLength","frequency1","frequency2","frequency_band"]
mapped_links_near_precip = []
precip_devices_field_names = ["name","lat","long","threshold"]
# maximum_perpendicular_distance = 10
link_ids_near_precip_csv = link_ids_near_precip_csv + ".csv"

In [45]:
def calc_equation(lat1, lang1, lat2, lang2):
    points = [(lat1, lang1), (lat2,lang2)]
    x_coords, y_coords = zip(*points)
    A = vstack([x_coords, ones(len(x_coords))]).T
    m, c = lstsq(A, y_coords)[0]
    return m,c

def calc_distance(lat1, lang1, lat2, lang2,lat_precip,long_precip):

    lat1= float(lat1)
    lat2 = float(lat2)
    lang1 = float(lang1)
    lang2 = float(lang2)
    start_cord = (lat_precip,long_precip)
    mid_point = ((lat1+lat2)/2,(lang1+lang2)/2)
    distance = geopy.distance.vincenty(start_cord, mid_point).km
    return distance
def path_distance(lat1,lang1,lat2,lang2):
    lat1= float(lat1)
    lat2 = float(lat2)
    lang1 = float(lang1)
    lang2 = float(lang2)
    start = (lat1,lang1)
    end = (lat2,lang2)
    return geopy.distance.vincenty(start, end).km    

In [46]:
precip_devices = []
with open(precip_devices_csv_file,"r") as precip_devices_csv:
    precip_devices_dict = csv.DictReader(precip_devices_csv,fieldnames=precip_devices_field_names)
    next(precip_devices_dict)
    for precip_device in precip_devices_dict:
        prep = {}
        prep['name'] = precip_device['name']
        prep['lat'] = float(precip_device['lat'])
        prep['long'] = float(precip_device['long'])
        prep["threshold"] = float(precip_device["threshold"])
        precip_devices.append(prep)
with open(mapped_links_csv_file,"r") as mapped_links_csv:
    mapped_links = csv.DictReader(mapped_links_csv, fieldnames=mapped_links_field_names)
    next(mapped_links)
    for link in mapped_links:
        for precip_device in precip_devices:
            mapped_link = {}
            distance = calc_distance(link["latitude1"],link["longitude1"],link["latitude2"],link["longitude2"],precip_device["lat"],precip_device["long"])
            if(distance<=precip_device["threshold"]):
                mapped_link["distance"] = distance
                mapped_link["nearest_precip_station"] = precip_device["name"]
                mapped_link["nearest_precip_lat"] = precip_device["lat"]
                mapped_link["nearest_precip_long"] = precip_device["long"]
                mapped_link["PathLength"] = path_distance(link["latitude1"],link["longitude1"],link["latitude2"],link["longitude2"])
                mapped_link["frequency1"] = link["frequency1"]
                mapped_link["frequency2"] = link["frequency2"]
                mapped_link["cell_tower_id1"] = link["cell_tower_id1"]
                mapped_link["cell_tower_id2"] = link["cell_tower_id2"]
                mapped_link["source1_name"] = link["source1_name"]
                mapped_link["source2_name"] = link["source2_name"]
                mapped_link["latitude1"] = link["latitude1"]
                mapped_link["latitude2"] = link["latitude2"]
                mapped_link["longitude1"] = link["longitude1"]
                mapped_link["longitude2"] = link["longitude2"]
                mapped_link["frequency_band"] = link["frequency_band"]
                mapped_link["district1"] = link["district1"]
                mapped_link["district2"] = link["district2"]
                mapped_link["id_1"] = link["id_1"]
                mapped_link["id_2"] = link["id_2"]
                mapped_links_near_precip.append(mapped_link)
                break
    with open(link_ids_near_precip_csv,"w") as processed_file_csv:
        dictionary_writer = csv.DictWriter(processed_file_csv,formatted_links_field_names)
        dictionary_writer.writeheader()
        mapped_links_near_precip = [dict(t) for t in set([tuple(d.items()) for d in mapped_links_near_precip])]
        mapped_links_near_precip_sorted = sorted(mapped_links_near_precip , key=lambda k: k['nearest_precip_station'])
        dictionary_writer.writerows(mapped_links_near_precip_sorted)